In [1]:
from requests import get
from bs4 import BeautifulSoup
import os

import pandas as pd
from datetime import date
import datetime

import preprocessing
import model
import wrangle

In [2]:
# airline = "aa"
# flight_number = "1988"
# flight_date = "08/02/2020"

In [3]:
airline = input("What is the airline? ")

What is the airline? AA


In [4]:
flight_number = input("What is the flight number? ")

What is the flight number? 2946


In [5]:
flight_date = input("When are your flying? ")

When are your flying? 08/02/2020


# Webscrapping

In [6]:
today = date.today()
print("Today's date:", today)

Today's date: 2020-07-22


In [7]:
month = today.month

In [8]:
day = today.day

In [9]:
year = today.year

In [10]:
datee = datetime.datetime.strptime(flight_date, "%m/%d/%Y")
datee

datetime.datetime(2020, 8, 2, 0, 0)

In [11]:
datee.day

2

In [12]:
url = f'https://www.flightstats.com/v2/flight-tracker/{airline}/{flight_number}?year={year}&month={month}&date={day}&flightId=1040676332'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
response = get(url, headers=headers)

In [13]:
soup = BeautifulSoup(response.content, 'html.parser')

In [14]:
airports = soup.find_all("a",class_="ticket__AirportLink-s1rrbl5o-10 cuIkvt")

In [15]:
dep_airport = airports[0].text
dep_airport

'ORD'

In [16]:
arr_airport = airports[1].text
arr_airport

'ATL'

In [17]:
crs_times = soup.find_all("div", class_="text-helper__TextHelper-s8bko4a-0 cCfBRT")

In [18]:
crs_dep_time = crs_times[0].text
crs_dep_time = int(crs_dep_time[0:5].replace(":",""))
crs_dep_time

1020

In [19]:
crs_arr_time = crs_times[2].text
crs_arr_time = int(crs_arr_time[0:5].replace(":", ""))
crs_arr_time

1326

# Preprocessing

In [20]:
features_for_modeling = ["op_carrier", "origin", "dest", "dest_airport_avg_delay", "carrier_avg_delay", "airport_avg_delay", "crs_dep_time", "crs_arr_time", "month", "day_of_week"]
target_variable = "is_delay"

In [21]:
# train_scaled, y_train, validate_scaled, y_validate, test_scaled, y_test = preprocessing.mvp_modeling_prep(modeling=True, features_for_modeling=features_for_modeling, target_variable=target_variable)

In [22]:
df = preprocessing.mvp_modeling_prep(features_for_modeling = features_for_modeling, target_variable = target_variable)

In [23]:
df.head()

,op_carrier,origin,dest,dest_airport_avg_delay,carrier_avg_delay,airport_avg_delay,crs_dep_time,crs_arr_time,month,day_of_week,is_delay
observation,,,,,,,,,,,
2018-01-01_UA_2429,UA,EWR,DEN,4.163647,6.260671,8.998358,1517,1745,1,0,False
2018-01-01_UA_2427,UA,LAS,SFO,9.162307,6.260671,4.242133,1115,1254,1,0,False
2018-01-01_UA_2424,UA,ORD,ALB,6.406991,6.260671,10.384382,630,922,1,0,True
2018-01-01_UA_2422,UA,ORD,OMA,3.662693,6.260671,10.384382,2241,14,1,0,False
2018-01-01_UA_2421,UA,IAH,LAS,3.966989,6.260671,3.795300,750,916,1,0,False


In [24]:
carrier_avg_delay = df[df.op_carrier == airline.upper()].carrier_avg_delay.mean()

In [25]:
dest_airport_avg_delay = df[df.dest == arr_airport].dest_airport_avg_delay.mean()

In [26]:
airport_avg_delay = df[df.origin == dep_airport].airport_avg_delay.mean()

In [27]:
prediction = pd.DataFrame({"carrier_avg_delay":carrier_avg_delay, "dest_airport_avg_delay": dest_airport_avg_delay, "airport_avg_delay":airport_avg_delay, "crs_dep_time": crs_dep_time, "crs_arr_time": crs_arr_time, "month": datee.month, "day_of_week":datee.day}, index=[f"{flight_date}_{airline.upper()}_{flight_number}"])

# Model

In [28]:
features_for_modeling = ["dest_airport_avg_delay", "carrier_avg_delay", "airport_avg_delay", "crs_dep_time", "crs_arr_time", "month", "day_of_week"]
target_variable = "is_delay"

In [29]:
df = wrangle.prep_flight_data()
df = preprocessing.to_date_time(df)
df = preprocessing.create_new_features(df)
df = preprocessing.create_target_variable(df)

df = df[df.origin == dep_airport]

features_for_modeling += ["observation"]
features_for_modeling += [target_variable]

df_modeling = df[features_for_modeling]

df_modeling = df_modeling.set_index("observation")


train, validate, test = preprocessing.split_data(df_modeling)

X_train = train.drop(columns=target_variable)
y_train = train[target_variable]
X_validate = validate.drop(columns=target_variable)
y_validate = validate[target_variable]
X_test = test.drop(columns=target_variable)
y_test = test[target_variable]

scaler, train_scaled, validate_scaled, test_scaled = preprocessing.min_max_scaler(X_train, X_validate, X_test)

In [30]:
knn, y_pred = model.run_knn(train_scaled, y_train, 3)

In [31]:
accuracy_score, matrix, report = model.accuracy_report(knn, y_pred, y_train)

print(accuracy_score)
print(matrix)

report

Accuracy on dataset: 0.77
       False  True 
False  86179  18499
True   23531  54512


,False,True,accuracy,macro avg,weighted avg
precision,0.785516,0.746627,0.769977,0.766072,0.768906
recall,0.823277,0.698487,0.769977,0.760882,0.769977
f1-score,0.803954,0.721755,0.769977,0.762854,0.768845
support,104678.000000,78043.000000,0.769977,182721.000000,182721.000000


In [32]:
knn.score(test_scaled, y_test)

0.5718876985592908

In [33]:
probability = knn.predict_proba(prediction)

In [34]:
print(f"The probability that your flight will get delayed is: {probability[0][1]:.0%}")

The probability that your flight will get delayed is: 67%
